In [1]:
import os
import gdown

def download_file_if_not_exists(file_url, output_path):
    """Скачивает файл с Google Drive, если он ещё не существует в указанной директории."""
    # Проверка наличия файла
    if os.path.exists(output_path):
        print(f"Файл '{output_path}' уже существует.")
    else:
        print(f"Файл '{output_path}' не найден. Начинаю загрузку...")
        gdown.download(file_url, output_path, quiet=False)
        print(f"Файл '{output_path}' успешно загружен.")

# Указываем URL и путь к файлу
file_url = 'https://drive.google.com/uc?id=1alondqI-2IHo__mYU7KQz4Ip8ytYGHXg'
output_file_name = 'wildberries_reviews.csv'  # Укажите реальное имя файла, которое хотите сохранить
output_path = os.path.join(os.getcwd(), output_file_name)  # Полный путь к файлу

download_file_if_not_exists(file_url, output_path)


Файл '/workspace/wildberries_reviews.csv' уже существует.


In [2]:
# !pip uninstall tensorflow tensorflow-gpu -y
# !pip uninstall torch torchvision torchaudio -y
# # Установка TensorFlow
# !pip install tensorflow==2.12.0

# # Установка PyTorch с поддержкой CUDA 12.1
# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
# !wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2404/x86_64/cuda-keyring_1.1-1_all.deb
# !dpkg -i cuda-keyring_1.1-1_all.deb
# !apt-get update
# !apt-get -y install cudnn
# !export PATH=/usr/local/cuda/bin:$PATH
# !export LD_LIBRARY_PATH=/usr/local/cuda/lib64:$LD_LIBRARY_PATH

In [1]:
# Установите и импортируйте модуль
!pip install -e /workspace/reviews_keywords/

Obtaining file:///workspace/reviews_keywords
  Preparing metadata (setup.py) ... done
  Attempting uninstall: reviews_keywords
    Found existing installation: reviews_keywords 0.1
    Uninstalling reviews_keywords-0.1:
      Successfully uninstalled reviews_keywords-0.1
  DEPRECATION: Legacy editable install of reviews_keywords==0.1 from file:///workspace/reviews_keywords (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for reviews_keywords


In [7]:
!pip install pytorch-crf
!pip install --upgrade deeppavlov

In [4]:
configs

Struct({'squad': {'squad_ru_convers_distilrubert_6L': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/squad_ru_convers_distilrubert_6L.json'),
         'squad_ru_convers_distilrubert_2L': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/squad_ru_convers_distilrubert_2L.json'),
         'squad_bert': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/squad_bert.json'),
         'qa_squad2_bert': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/qa_squad2_bert.json'),
         'squad_ru_bert': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/squad_ru_bert.json'),
         'qa_nq_psgcls_bert': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/qa_nq_psgcls_bert.json'),
         'qa_multisberquad_bert': PosixPath('/opt/conda/lib/python3.10/site-packages/deeppavlov/configs/squad/qa_multisberquad_bert.json')},
        'ranking': {'rel_r

In [7]:
from deeppavlov import build_model, configs
from spacy.tokens import Doc, Span
import spacy
from spacy.language import Language

# Загрузите NER модель DeepPavlov
ner_model = build_model(configs.ner.ner_rus_convers_distilrubert_6L, download=True)

@Language.component("deeppavlov_ner")
def deeppavlov_ner(doc: Doc) -> Doc:
    tokens = [token.text for token in doc]
    entities = ner_model([tokens])[1][0]
    
    # Учитываем структуру данных, возвращаемых моделью
    spans = [Span(doc, start, end, label=label) for start, end, label, _ in entities]
    
    doc.ents = spans
    return doc

# Интеграция кастомного компонента в spaCy pipeline
nlp = spacy.load("ru_core_news_lg")
nlp.add_pipe("deeppavlov_ner", after="ner")

# Пример текста
doc = nlp("Владимир Путин встретился с министром иностранных дел Сергеем Лавровым.")
for ent in doc.ents:
    print(ent.text, ent.label_)


2024-08-14 09:07:11.913 INFO in 'deeppavlov.download'['download'] at line 138: Skipped http://files.deeppavlov.ai/v1/ner/ner_rus_conversational_distilrubert_6L.tar.gz download because of matching hashes
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/distilrubert-base-cased-conversational and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: too many values to unpack (expected 4)

In [1]:
!pip install fasttext

  Using cached fasttext-0.9.3.tar.gz (73 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=321786 sha256=3189ed9e69ed5946686095019c6b3d33711d4a679e1ce5567d964d00a5fb4f3a
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [6]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from numpy.linalg import norm  # Импортируем norm из numpy.linalg


# Загрузка модели и токенизатора
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained('DeepPavlov/rubert-base-cased')

def get_bert_vector(text):
    inputs = tokenizer(text, return_tensors='pt')
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).detach().numpy().flatten()

# Пример для сравнения двух категорий
category_1 = "автотовары"
category_2 = "автотовары"

vector1 = get_bert_vector(category_1)
vector2 = get_bert_vector(category_2)

# Проверка на нулевые вектора
if norm(vector1) == 0 or norm(vector2) == 0:
    print("Один из векторов является нулевым. Невозможно вычислить сходство.")
else:
    similarity = np.dot(vector1, vector2) / (norm(vector1) * norm(vector2))
    print(f"Cosine Similarity: {similarity}")


Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Cosine Similarity: 1.0


In [8]:
print("Vector for category 1:", doc1.vector)
print("Vector for category 2:", doc2.vector)


Vector for category 1: [ 3.57762963e-01 -2.17684448e-01  1.86183259e-01 -2.10745826e-01
 -1.36316165e-01  9.17311683e-02  1.85587510e-01 -1.95953846e-02
 -5.31683862e-02 -1.79790199e-01  2.14173034e-01 -7.57782673e-03
 -1.35014445e-01 -1.05054311e-01 -4.79594618e-02  1.30265370e-01
  9.74741057e-02  7.03092664e-02 -1.18495584e-01  1.38237640e-01
  2.42913768e-01  1.15407305e-02 -1.02995358e-01  5.22607565e-03
 -1.58066023e-02  2.66199023e-01  1.41095191e-01  3.32794875e-01
  2.46084377e-01 -6.71619847e-02 -2.32162356e-01 -1.96225092e-01
  3.58834594e-01  3.17079686e-02  4.61726673e-02 -1.38651028e-01
 -2.91880760e-02 -1.15724184e-01  1.73585117e-02  2.52240658e-01
  2.93866098e-01 -1.03361793e-01  3.39522749e-01  6.06537946e-02
 -7.01813251e-02  2.14154780e-01  3.25398386e-01  7.24065900e-02
  6.66890666e-02  4.40435112e-03  4.45913784e-02  1.03087097e-01
 -1.51141062e-01  1.95404455e-01  7.03163818e-02 -1.83241367e-02
 -2.37746954e-01 -6.20814152e-02 -1.37596980e-01 -1.64018437e-01
  

In [1]:
import pandas as pd
from reviews_keywords.main import *

# Загрузите ваши данные
data = pd.read_csv('wildberries_reviews.csv', nrows=30000).iloc[-100:-1][::-1]


# Укажите категории, которые хотите проанализировать
categories = data.iloc[-1].category  # Или оставьте пустым для анализа всех категорий
# Проведите анализ
# all_aspect_df = analyze_reviews(data, categories)
all_aspect_df = analyze_reviews(data)

from IPython.display import display

all_aspect_df


2024-08-14 08:37:02.321658: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-14 08:37:02.352455: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 08:37:02.982741: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


torch.cuda.is_available() - True
torch.cuda.is_available() - True


Processing docs: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83/83 [00:01<00:00, 60.61it/s]


,category,aspect_group_type,product,aspect,aspect_add,grouped_aspects,count,positive,negative,neutral,descriptions,characteristics,original_combinations,total_characteristics_count,rating_positive,rating_negative,sentiment,key_thought_sumy,agreed_phrases
0,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,Тип мебели,None,,25,11,3,0,Аспект 'Тип мебели' оценивается клиентами в ос...,"самый, 5, знать, нравиться, порадовать, упаков...","[Пахнет хорошо, флакон полный доверху., Лёгкий...",25,18,7,positive,Очень нравится аромат Ну и самый главный косяк...,"[содержимое видное, содержимое разлить, содерж..."
1,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,товар,приобретать,,11,6,2,0,Аспект 'Товар' оценивается клиентами в основно...,"самый, хотеть, надёжный, советовать, маслянист...","[Советую, рекомендую, желаю приобретать., Аром...",15,6,5,positive,Буду заказывать еще однозначно рекомендую Так ...,"[приобретать желать приобретать, приобретать с..."
2,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,запах,None,,1,1,0,0,Аспект 'запах' оценивается клиентами в основно...,"лёгкий, резкий, мужской","[Лёгкий, не резкий мужской запах.]",3,1,0,positive,"Лёгкий, не резкий мужской запах.","[запах лёгкий, запах резкий, запах мужской, за..."
3,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,товар,супер,,5,5,0,0,Аспект 'супер' оценивается клиентами в основно...,"прийти, упаковать","[Запах просто супер., Качество супер, пришел в...",2,5,0,positive,"Качество супер, пришел в целости и сохранности...","[супер упаковать супер, супер прийти супер]"
4,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,товар,нереальный,,1,1,0,0,Аспект 'нереальный' оценивается клиентами в ос...,"пахнуть, прийти","[Пришло быстро, запах нереальный в машине пахн...",2,1,0,positive,"Пришло быстро, запах нереальный в машине пахне...","[нереальный пахнуть нереальный, нереальный при..."
5,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,бутылей,None,,1,0,1,0,Аспект 'бутылей' оценивается клиентами в основ...,"забрать, пустой","[Забрал заказ 12 июня, 5 июля бутылей уже был ...",2,0,1,negative,"Забрал заказ 12 июня, 5 июля бутылей уже был п...","[бутылей пустой, бутылей забрать, бутылей пуст..."
6,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,отверстие,None,,1,0,1,0,Аспект 'отверстие' оценивается клиентами в осн...,"делать, забрать","[Забрал заказ 12 июня, 5 июля бутылей уже был ...",2,0,1,negative,"Забрал заказ 12 июня, 5 июля бутылей уже был п...","[отверстие делать, отверстие забрать, отверсти..."
7,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,товар,бояться,,1,1,0,0,Аспект 'бояться' оценивается клиентами в основ...,"случиться, хотеть","[Так же хочу отметить надежную упаковку, можно...",2,1,0,positive,"Так же хочу отметить надежную упаковку, можно ...","[бояться случиться бояться, бояться хотеть боя..."
8,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,товар,закрашивать,,1,0,1,0,Аспект 'закрашивать' оценивается клиентами в о...,белый,[Чем теперь закрашивать белые пятна?],1,0,1,negative,Чем теперь закрашивать белые пятна?,"[закрашивать белый закрашивать, закрашивать бе..."
9,/Автотовары/Аксессуары в салон и багажник/Аром...,None,Пахнет и Точка / Ароматизатор в машину автопар...,коробочка,None,,1,1,0,0,Аспект 'коробочка' оценивается клиентами в осн...,упаковать,"[Упаковано замечательно, коробочка супер, дере...",1,1,0,positive,"Упаковано замечательно, коробочка супер, дерев...","[коробочка упаковать, коробочка упаковать]"


In [5]:
all_aspect_df.iloc[-1]["descriptions"]

"Аспект 'прочный' оценивается клиентами в основном положительно. Например: «Очень прочный материал.»."

In [6]:
category_product_summary = create_category_product_summary(all_aspect_df)
recommendations = recommend_products(category_product_summary)
pricing_recommendations = recommend_pricing(category_product_summary)
product_cards = generate_product_card(category_product_summary, all_aspect_df)

display("Рекомендации для популярных товаров:")
display(recommendations['popular'])

display("\nРекомендации для менее популярных товаров:")
display(recommendations['less_popular'])

display("\nРекомендации по изменению цены:")
display(pricing_recommendations)

display("\nКарточки товаров:")
display(product_cards)

'Рекомендации для популярных товаров:'

{'product': 'ВЫРУЧАЙКА / Антибукс Противобуксовочные траки для автомобиля',
 'positive_ratio': 0.8482142857142857,
 'rating_positive': 96,
 'rating_negative': 16,
 'positive_aspects': 'вещь оказаться, выручать проверять выручать, пользоваться давать пользоваться, пользоваться узкий пользоваться, воспользоваться купить воспользоваться, воспользоваться прийтись воспользоваться, внешне знать внешне, внешне повести внешне, огонь знать, огонь повести',
 'negative_aspects': 'проверить решить проверить, сёгуне буксовать, колёс вылетать, толку вылетать, хранён прийти хранён, выручайся стачиваться выручайся, специально застрять специально, тахта застрять, сёгуне маленьком, практически помогать практически',
 'positive_empty_aspects': [],
 'negative_empty_aspects': []}

'\nРекомендации для менее популярных товаров:'

{'product': 'N/A',
 'positive_ratio': 'N/A',
 'rating_positive': 'N/A',
 'rating_negative': 'N/A',
 'positive_aspects': 'N/A',
 'negative_aspects': 'N/A',
 'positive_empty_aspects': 'N/A',
 'negative_empty_aspects': 'N/A'}

'\nРекомендации по изменению цены:'

,category,product,price_change_recommendation
0,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,No change
1,/Автотовары/OFFroad,URAL SKLAD / Лебедка электрическая 4000 lbs 18...,No change
2,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3000lb 13...,No change


'\nКарточки товаров:'

,category,product,title,description,key_features,advantages
0,/Автотовары/OFFroad,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,ВЫРУЧАЙКА / Антибукс Противобуксовочные траки ...,Этот товар имеет следующие положительные харак...,", застрять, знать, повести, выглядеть, вылетать","вещь оказаться, выручать проверять выручать, п..."
1,/Автотовары/OFFroad,URAL SKLAD / Лебедка электрическая 4000 lbs 18...,URAL SKLAD / Лебедка электрическая 4000 lbs 18...,Этот товар имеет следующие положительные харак...,"работать, тянуть, сгореть,","нормально работать нормально, отлично тянуть о..."
2,/Автотовары/OFFroad,Shtapler / Лебедка электрическая 12v 3000lb 13...,Shtapler / Лебедка электрическая 12v 3000lb 13...,Этот товар имеет следующие положительные харак...,", купить, отломиться, проверять, пойти","быстро прийти быстро, установка купить, сердит..."


In [7]:
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# Загрузка модели Spacy для русского языка
nlp = spacy.load("ru_core_news_lg")

# Пример данных
reviews = [
    "Работает хорошо.",
    "Пришло быстро, все целое на вид. Завтра буду испытывать.",
    "Купил на квадр для поднятия отвала, установка заняла час👍"
]

categories = [
    "/Автотовары/OFFroad",
    "Main"
]

# Векторизация текста с использованием Spacy
def vectorize_text_spacy(text):
    doc = nlp(text)
    return doc.vector

# Векторизация категорий
category_embeddings = [vectorize_text_spacy(cat) for cat in categories]

# Сопоставление каждого отзыва с категорией
for review in reviews:
    review_vector = vectorize_text_spacy(review)
    
    # Вычисление косинусного сходства между отзывом и каждой категорией
    similarities = cosine_similarity([review_vector], category_embeddings)
    
    # Поиск наибольшего сходства
    max_similarity_index = similarities.argmax()
    matched_category = categories[max_similarity_index]
    
    print(f"Отзыв: \"{review}\" наиболее близок к категории: {matched_category} (сходство: {similarities[0][max_similarity_index]:.2f})")


Отзыв: "Работает хорошо." наиболее близок к категории: /Автотовары/OFFroad (сходство: 0.00)
Отзыв: "Пришло быстро, все целое на вид. Завтра буду испытывать." наиболее близок к категории: /Автотовары/OFFroad (сходство: 0.00)
Отзыв: "Купил на квадр для поднятия отвала, установка заняла час👍" наиболее близок к категории: /Автотовары/OFFroad (сходство: 0.00)
